In [5]:
import warnings
warnings.filterwarnings("ignore")

import os
import cv2
import mediapipe as mp
import numpy as np
import time
import winsound
import csv
from datetime import datetime
from collections import deque

SIDE_LEAN_LIMIT = 0.05
NECK_DROP_LIMIT = 0.20
CLOSE_LIMIT = 1.45
ISIK_LIMITI = 60
SES_ACIK = True


SUSTAIN_FRAMES = 12          
MIN_LM_VIS = 0.55            


RATIO_LIMIT = 1.12           
FACEZ_DELTA = 0.10          


EVIDENCE_COOLDOWN = {
    "YAKIN_MESAFE": 15,
    "SKOLYOZ_RISKI": 20,
    "TELEFON_BOYNU": 20
}


DEBUG_MODE = False


ERROR_LABELS = {
    "YAKIN_MESAFE": "Cok_Yakin",
    "SKOLYOZ_RISKI": "Yana_Egilme",
    "TELEFON_BOYNU": "Boyun_Egilme"
}
ERROR_TITLES_TR = {
    "YAKIN_MESAFE": "COK YAKINSIN!",
    "SKOLYOZ_RISKI": "YANA EGILME VAR!",
    "TELEFON_BOYNU": "BOYUN EGME VAR!"
}


KANIT_KLASORU = "Kanit_Fotograflari"
RAPOR_DOSYASI = "Detayli_Postur_Raporu.csv"

if not os.path.exists(KANIT_KLASORU):
    os.makedirs(KANIT_KLASORU)

if not os.path.exists(RAPOR_DOSYASI):
    with open(RAPOR_DOSYASI, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Tarih", "Saat", "Durum", "Detay", "Skor"])


mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)


calibrated = False
ref_eye_dist = 0.0
ref_shoulder_slope = 0.0
ref_nose_dist = 0.0
ref_shoulder_width = 0.0
ref_eye_sh_ratio = 0.0
ref_face_z = 0.0


start_time = 0
monitored_frames = 0
good_frames = 0
last_beep = 0


buffer_eye = deque(maxlen=10)
buffer_slope = deque(maxlen=10)
buffer_nose = deque(maxlen=10)
buffer_shw = deque(maxlen=10)
buffer_facez = deque(maxlen=10)

graph_data = deque(maxlen=200)


error_streak = {"YAKIN_MESAFE": 0, "SKOLYOZ_RISKI": 0, "TELEFON_BOYNU": 0}
last_evidence_time = {"YAKIN_MESAFE": 0, "SKOLYOZ_RISKI": 0, "TELEFON_BOYNU": 0}


def draw_hud(img, x, y, w, h, title, color=(30, 30, 30)):
    overlay = img.copy()
    cv2.rectangle(overlay, (x, y), (x + w, y + h), color, -1)
    cv2.addWeighted(overlay, 0.7, img, 0.3, 0, img)
    cv2.rectangle(img, (x, y), (x + w, y + h), (100, 100, 100), 1)
    if title:
        cv2.putText(img, title, (x + 12, y + 26), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (220, 220, 220), 1)

def draw_card(img, x, y, w, h, title, lines, border_color=(120,120,120), header_color=(255,255,255)):
    overlay = img.copy()
    cv2.rectangle(overlay, (x, y), (x + w, y + h), (15, 15, 15), -1)
    cv2.addWeighted(overlay, 0.75, img, 0.25, 0, img)

    cv2.rectangle(img, (x, y), (x + w, y + h), border_color, 2)
    cv2.putText(img, title, (x + 12, y + 26), cv2.FONT_HERSHEY_SIMPLEX, 0.65, header_color, 2)

    ty = y + 55
    for t in lines:
        cv2.putText(img, t, (x + 12, ty), cv2.FONT_HERSHEY_SIMPLEX, 0.52, (230,230,230), 1)
        ty += 22

def log_to_excel(durum, detay, skor=None, metrics=None):
    now = datetime.now()
    metrics_str = ""
    if metrics:
        metrics_str = " | ".join([f"{k}:{v}" for k, v in metrics.items()])
    with open(RAPOR_DOSYASI, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            now.strftime("%Y-%m-%d"),
            now.strftime("%H:%M:%S"),
            durum,
            detay if not metrics_str else f"{detay} || {metrics_str}",
            "" if skor is None else skor
        ])

def save_evidence(frame, error_type):
    label = ERROR_LABELS.get(error_type, error_type)
    filename = f"{KANIT_KLASORU}/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{label}.jpg"
    cv2.imwrite(filename, frame)
    return filename

def get_brightness(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    return float(np.mean(hsv[:, :, 2]))

def annotate_evidence(img, error_code, score, elapsed_s, brightness, ref_vals, cur_vals):
    overlay = img.copy()
    cv2.rectangle(overlay, (20, 20), (820, 220), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.65, img, 0.35, 0, img)

    m, s = divmod(elapsed_s, 60)
    title = ERROR_TITLES_TR.get(error_code, error_code)

    lines = [
        f"HATA: {title}",
        f"Skor: {score}/100   Sure: {m:02d}:{s:02d}   Isik: {brightness:.1f}",
    ]

    if DEBUG_MODE:
        lines += [
            f"eye_dist: {cur_vals['eye']:.2f} | limit: {ref_vals['eye_limit']:.2f}",
            f"ratio: {cur_vals['ratio']:.3f} | limit: {ref_vals['ratio_limit']:.3f}",
            f"slope: {cur_vals['slope']:.4f} | limit: {ref_vals['slope_limit']:.4f}",
            f"nose_dist: {cur_vals['nose']:.2f} | limit: {ref_vals['nose_limit']:.2f}",
            f"face_z: {cur_vals['facez']:.3f} | limit: {ref_vals['facez_limit']:.3f}",
        ]
    else:
        if error_code == "YAKIN_MESAFE":
            lines.append("Oneri: Biraz geri cekil.")
        elif error_code == "SKOLYOZ_RISKI":
            lines.append("Oneri: Omuzlarini hizala, dik otur.")
        elif error_code == "TELEFON_BOYNU":
            lines.append("Oneri: Ceneni kaldir, karsiya bak.")

    y = 55
    for i, t in enumerate(lines):
        cv2.putText(img, t, (40, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7 if i == 0 else 0.55,
                    (0, 255, 255) if i == 0 else (255, 255, 255),
                    2 if i == 0 else 1)
        y += 26

def clamp01(x):
    return max(0.0, min(1.0, float(x)))

def draw_status_dot(img, x, y, state):
    if state == "good":
        c = (0, 255, 0)
    elif state == "warn":
        c = (0, 165, 255)
    else:
        c = (0, 0, 255)
    cv2.circle(img, (x, y), 10, c, -1)
    cv2.circle(img, (x, y), 10, (40, 40, 40), 2)

def draw_bar(img, x, y, w, h, pct01, state):
    pct01 = clamp01(pct01)
    cv2.rectangle(img, (x, y), (x + w, y + h), (60, 60, 60), -1)
    fill_w = int(w * pct01)

    if state == "good":
        c = (0, 255, 0)
    elif state == "warn":
        c = (0, 165, 255)
    else:
        c = (0, 0, 255)

    cv2.rectangle(img, (x, y), (x + fill_w, y + h), c, -1)
    cv2.rectangle(img, (x, y), (x + w, y + h), (110, 110, 110), 1)

def state_from_score(sc):
    if sc < 0.95:
        return "good"
    if sc < 1.05:
        return "warn"
    return "bad"

print(f"✅ SİSTEM HAZIR. Kanıtlar '{KANIT_KLASORU}' klasörüne kaydedilecek.")


score = 100
key = -1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1024, 768))
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False
    results = pose.process(image_rgb)
    image_rgb.flags.writeable = True
    image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    h, w, _ = image.shape
    brightness = get_brightness(frame)

    try:
        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark

            l_sh_lm = lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            r_sh_lm = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            l_eye_lm = lm[mp_pose.PoseLandmark.LEFT_EYE.value]
            r_eye_lm = lm[mp_pose.PoseLandmark.RIGHT_EYE.value]
            nose_lm = lm[mp_pose.PoseLandmark.NOSE.value]

            if (l_sh_lm.visibility < MIN_LM_VIS or r_sh_lm.visibility < MIN_LM_VIS or
                l_eye_lm.visibility < MIN_LM_VIS or r_eye_lm.visibility < MIN_LM_VIS or
                nose_lm.visibility < MIN_LM_VIS):

                cv2.putText(image, "Landmark guveni dusuk (bekleniyor)...", (30, h - 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        else:
            cv2.putText(image, "Vucut algilanamadi. Kameraya bakar misin?", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        if not results.pose_landmarks:
            cv2.putText(image, "[Q] CIKIS", (w - 150, h - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (220, 220, 220), 2)

            cv2.imshow('PRO POSTUR ANALIZI', image)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            continue

        l_sh = np.array([l_sh_lm.x * w, l_sh_lm.y * h])
        r_sh = np.array([r_sh_lm.x * w, r_sh_lm.y * h])
        nose = np.array([nose_lm.x * w, nose_lm.y * h])
        l_eye = np.array([l_eye_lm.x * w, l_eye_lm.y * h])
        r_eye = np.array([r_eye_lm.x * w, r_eye_lm.y * h])

        raw_eye_dist = float(np.linalg.norm(l_eye - r_eye))
        raw_shoulder_width = float(np.linalg.norm(l_sh - r_sh))
        raw_slope = float(abs(l_sh[1] - r_sh[1]) / h)
        sh_mid = (l_sh + r_sh) / 2.0
        raw_nose_dist = float(abs(sh_mid[1] - nose[1]))
        raw_face_z = float((l_eye_lm.z + r_eye_lm.z + nose_lm.z) / 3.0)

        buffer_eye.append(raw_eye_dist)
        buffer_shw.append(raw_shoulder_width)
        buffer_slope.append(raw_slope)
        buffer_nose.append(raw_nose_dist)
        buffer_facez.append(raw_face_z)

        avg_eye = sum(buffer_eye) / len(buffer_eye)
        avg_shw = max(1e-6, sum(buffer_shw) / len(buffer_shw))
        avg_slope = sum(buffer_slope) / len(buffer_slope)
        avg_nose = sum(buffer_nose) / len(buffer_nose)
        avg_facez = sum(buffer_facez) / len(buffer_facez)

        avg_ratio = avg_eye / avg_shw
        graph_data.append(avg_slope * 1000)

       
        if not calibrated:
            draw_hud(image, 0, 0, w, h, "", (0, 0, 0))
            cx, cy = w // 2, h // 2
            cv2.putText(image, "KALIBRASYON", (cx - 120, cy - 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.putText(image, "Dik Dur ve [K] Tusuna Bas", (cx - 220, cy + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 1)

            cv2.line(image, (int(l_sh[0]), int(l_sh[1])), (int(r_sh[0]), int(r_sh[1])), (0, 255, 255), 2)
            cv2.putText(image, "Cikis: [Q]", (30, h - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.65, (220, 220, 220), 2)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            cv2.imshow('PRO POSTUR ANALIZI', image)
            key = cv2.waitKey(1) & 0xFF

            if key == ord('k'):
                ref_eye_dist = avg_eye
                ref_shoulder_slope = avg_slope
                ref_nose_dist = avg_nose
                ref_shoulder_width = avg_shw
                ref_eye_sh_ratio = avg_ratio
                ref_face_z = avg_facez

                calibrated = True
                start_time = time.time()
                monitored_frames = 0
                good_frames = 0

                for k2 in error_streak:
                    error_streak[k2] = 0
                    last_evidence_time[k2] = 0

                log_to_excel("BILGI", "Oturum Baslatildi (Kalibrasyon tamam)", skor=None, metrics={
                    "ref_eye": f"{ref_eye_dist:.2f}",
                    "ref_shw": f"{ref_shoulder_width:.2f}",
                    "ref_ratio": f"{ref_eye_sh_ratio:.3f}",
                    "ref_slope": f"{ref_shoulder_slope:.4f}",
                    "ref_nose": f"{ref_nose_dist:.2f}",
                    "ref_facez": f"{ref_face_z:.3f}",
                })
                winsound.Beep(1000, 200)

            if key == ord('q'):
                break

            continue

       
        monitored_frames += 1

        eye_trigger = avg_eye > (ref_eye_dist * CLOSE_LIMIT)
        ratio_trigger = avg_ratio > (ref_eye_sh_ratio * RATIO_LIMIT)
        facez_trigger = avg_facez < (ref_face_z - FACEZ_DELTA)

        is_close = eye_trigger and (ratio_trigger or facez_trigger)
        is_leaning = avg_slope > (ref_shoulder_slope + SIDE_LEAN_LIMIT)
        is_neck_bad = avg_nose < (ref_nose_dist * (1 - NECK_DROP_LIMIT))

        status = "MUKEMMEL"
        sub_status = "Boyle devam et"
        color_main = (0, 255, 0)
        error_code = None

        if is_close:
            status = "COK YAKIN!"
            sub_status = "Geri cekil"
            color_main = (0, 0, 255)
            error_code = "YAKIN_MESAFE"
            if SES_ACIK and (time.time() - last_beep > 1.5):
                winsound.Beep(3000, 120)
                last_beep = time.time()

        elif is_leaning:
            status = "YANA EGILME"
            sub_status = "Duzel"
            color_main = (0, 0, 255)
            error_code = "SKOLYOZ_RISKI"
            cv2.line(image, (int(l_sh[0]), int(l_sh[1])), (int(r_sh[0]), int(r_sh[1])), (0, 0, 255), 4)
            if SES_ACIK and (time.time() - last_beep > 1.5):
                winsound.Beep(500, 200)
                last_beep = time.time()

        elif is_neck_bad:
            status = "BOYUN EGME"
            sub_status = "Karsiya bak"
            color_main = (0, 165, 255)
            error_code = "TELEFON_BOYNU"
            cv2.line(image, (int(nose[0]), int(nose[1])), (int(sh_mid[0]), int(sh_mid[1])), (0, 0, 255), 4)
            if SES_ACIK and (time.time() - last_beep > 2):
                winsound.Beep(2000, 150)
                last_beep = time.time()
        else:
            good_frames += 1

        score = int((good_frames / monitored_frames) * 100) if monitored_frames > 0 else 100
        elapsed = int(time.time() - start_time)
        m, s = divmod(elapsed, 60)

       
        for k2 in error_streak.keys():
            if error_code == k2:
                error_streak[k2] += 1
            else:
                error_streak[k2] = 0

        if error_code:
            cooldown = EVIDENCE_COOLDOWN.get(error_code, 15)
            can_save = (error_streak[error_code] >= SUSTAIN_FRAMES) and \
                       (time.time() - last_evidence_time[error_code] >= cooldown)

            if can_save:
                evidence_img = image.copy()

                ref_vals = {
                    "eye_limit": ref_eye_dist * CLOSE_LIMIT,
                    "ratio_limit": ref_eye_sh_ratio * RATIO_LIMIT,
                    "slope_limit": ref_shoulder_slope + SIDE_LEAN_LIMIT,
                    "nose_limit": ref_nose_dist * (1 - NECK_DROP_LIMIT),
                    "facez_limit": ref_face_z - FACEZ_DELTA
                }
                cur_vals = {
                    "eye": avg_eye,
                    "ratio": avg_ratio,
                    "slope": avg_slope,
                    "nose": avg_nose,
                    "facez": avg_facez
                }

                annotate_evidence(
                    evidence_img, error_code,
                    score=score,
                    elapsed_s=elapsed,
                    brightness=brightness,
                    ref_vals=ref_vals,
                    cur_vals=cur_vals
                )

                file_path = save_evidence(evidence_img, error_code)
                print(f"📸 Kanıt Kaydedildi: {file_path}")

                log_to_excel(
                    "HATA",
                    f"{ERROR_LABELS.get(error_code, error_code)} (stabil: {error_streak[error_code]} frame)",
                    skor=score,
                    metrics={
                        "avg_eye": f"{avg_eye:.2f}",
                        "avg_shw": f"{avg_shw:.2f}",
                        "avg_ratio": f"{avg_ratio:.3f}",
                        "avg_slope": f"{avg_slope:.4f}",
                        "avg_nose": f"{avg_nose:.2f}",
                        "avg_facez": f"{avg_facez:.3f}",
                        "brightness": f"{brightness:.1f}",
                        "file": os.path.basename(file_path)
                    }
                )

                last_evidence_time[error_code] = time.time()

        
        draw_hud(image, 20, 20, 420, 210, "DURUS GENEL")
        cv2.putText(image, status, (40, 78), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color_main, 2)
        cv2.putText(image, sub_status, (40, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 1)
        cv2.putText(image, f"Skor: {score}/100", (40, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.72, (0, 255, 0), 2)
        cv2.putText(image, f"Sure: {m:02d}:{s:02d}", (40, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.62, (255, 255, 0), 1)

        if brightness < ISIK_LIMITI:
            cv2.putText(image, "ORTAM KARANLIK!", (40, 205),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 255), 1)

       
        card_w = 360
        card_h = 140   
        gap = 14
        rx = w - card_w - 20
        y0 = 20

        PAD_X = 16
        ROW1_Y = 52
        BAR_Y  = 78
        MSG_Y  = 116
        DOT_X = rx + PAD_X + 6
        DOT_Y_OFFSET = 5

        TEXT_SCALE = 0.58
        PCT_SCALE = 0.62
        DBG_SCALE = 0.42

      
        close_score_eye = avg_eye / (ref_eye_dist * CLOSE_LIMIT)
        close_score_ratio = avg_ratio / (ref_eye_sh_ratio * RATIO_LIMIT)
        close_score = max(close_score_eye, close_score_ratio)
        close_state = state_from_score(close_score)
        close_pct = clamp01((close_score - 0.7) / 0.6)

        cy = y0
        draw_card(image, rx, cy, card_w, card_h, "MESAFE", [], border_color=(120,120,120), header_color=(255,255,255))
        draw_status_dot(image, DOT_X, cy + ROW1_Y - DOT_Y_OFFSET, close_state)
        cv2.putText(image, "Yakinsin mi?", (rx + PAD_X + 26, cy + ROW1_Y),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SCALE, (240,240,240), 1)
        cv2.putText(image, f"{int(close_pct*100)}%", (rx + card_w - 78, cy + ROW1_Y),
                    cv2.FONT_HERSHEY_SIMPLEX, PCT_SCALE, (240,240,240), 2)
        draw_bar(image, rx + PAD_X, cy + BAR_Y, card_w - 2*PAD_X, 16, close_pct, close_state)
        msg = "Normal mesafe" if close_state == "good" else ("Biraz geri cekil" if close_state == "warn" else "Geri cekil!")
        cv2.putText(image, msg, (rx + PAD_X, cy + MSG_Y),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SCALE, (240,240,240), 1)

        if DEBUG_MODE:
            cv2.putText(image,
                        f"DBG eye:{avg_eye:.1f}/{ref_eye_dist*CLOSE_LIMIT:.1f} ratio:{avg_ratio:.3f}/{ref_eye_sh_ratio*RATIO_LIMIT:.3f} facez:{avg_facez:.3f}",
                        (rx + PAD_X, cy + card_h - 6), cv2.FONT_HERSHEY_SIMPLEX, DBG_SCALE, (180,180,180), 1)

       
        lean_score = avg_slope / (ref_shoulder_slope + SIDE_LEAN_LIMIT)
        lean_state = state_from_score(lean_score)
        lean_pct = clamp01((lean_score - 0.7) / 0.6)

        cy = y0 + card_h + gap
        draw_card(image, rx, cy, card_w, card_h, "OMUZ", [], border_color=(120,120,120), header_color=(255,255,255))
        draw_status_dot(image, DOT_X, cy + ROW1_Y - DOT_Y_OFFSET, lean_state)
        cv2.putText(image, "Denge", (rx + PAD_X + 26, cy + ROW1_Y),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SCALE, (240,240,240), 1)
        cv2.putText(image, f"{int(lean_pct*100)}%", (rx + card_w - 78, cy + ROW1_Y),
                    cv2.FONT_HERSHEY_SIMPLEX, PCT_SCALE, (240,240,240), 2)
        draw_bar(image, rx + PAD_X, cy + BAR_Y, card_w - 2*PAD_X, 16, lean_pct, lean_state)
        msg = "Omuzlar duz" if lean_state == "good" else ("Omuzlarini hizala" if lean_state == "warn" else "Yana egilme var!")
        cv2.putText(image, msg, (rx + PAD_X, cy + MSG_Y),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SCALE, (240,240,240), 1)

        if DEBUG_MODE:
            cv2.putText(image,
                        f"DBG slope:{avg_slope:.4f}/{(ref_shoulder_slope+SIDE_LEAN_LIMIT):.4f}",
                        (rx + PAD_X, cy + card_h - 6), cv2.FONT_HERSHEY_SIMPLEX, DBG_SCALE, (180,180,180), 1)

        neck_limit = ref_nose_dist * (1 - NECK_DROP_LIMIT)
        neck_score = neck_limit / max(1e-6, avg_nose)
        neck_state = state_from_score(neck_score)
        neck_pct = clamp01((neck_score - 0.7) / 0.6)

        cy = y0 + 2*(card_h + gap)
        draw_card(image, rx, cy, card_w, card_h, "BOYUN", [], border_color=(120,120,120), header_color=(255,255,255))
        draw_status_dot(image, DOT_X, cy + ROW1_Y - DOT_Y_OFFSET, neck_state)
        cv2.putText(image, "Riski", (rx + PAD_X + 26, cy + ROW1_Y),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SCALE, (240,240,240), 1)
        cv2.putText(image, f"{int(neck_pct*100)}%", (rx + card_w - 78, cy + ROW1_Y),
                    cv2.FONT_HERSHEY_SIMPLEX, PCT_SCALE, (240,240,240), 2)
        draw_bar(image, rx + PAD_X, cy + BAR_Y, card_w - 2*PAD_X, 16, neck_pct, neck_state)
        msg = "Bas dik" if neck_state == "good" else ("Ceneni kaldir" if neck_state == "warn" else "Boyun egme var!")
        cv2.putText(image, msg, (rx + PAD_X, cy + MSG_Y),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SCALE, (240,240,240), 1)

        if DEBUG_MODE:
            cv2.putText(image,
                        f"DBG nose:{avg_nose:.1f}/{neck_limit:.1f}",
                        (rx + PAD_X, cy + card_h - 6), cv2.FONT_HERSHEY_SIMPLEX, DBG_SCALE, (180,180,180), 1)

       
        if DEBUG_MODE:
            cv2.putText(image, "DEBUG: ON (D ile kapat)", (30, h - 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        else:
            cv2.putText(image, "DEBUG: OFF (D ile ac)", (30, h - 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (160, 160, 160), 2)

       
        gx, gy, gw, gh = 20, h - 150, 420, 130
        draw_hud(image, gx, gy, gw, gh, "OMUZ EGRISI (SLOPE)")
        if len(graph_data) > 1:
            pts = []
            for i, val in enumerate(graph_data):
                ny = max(0, min(1, val / 100))
                pts.append((gx + int(i * (gw / 200)), (gy + gh) - int(ny * gh)))
            for i in range(1, len(pts)):
                cv2.line(image, pts[i - 1], pts[i], (0, 255, 255), 2)

        cv2.putText(image, "[D] DEBUG   [R] SIFIRLA   [Q] CIKIS", (w - 470, h - 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (220, 220, 220), 2)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    except Exception:
        pass

    cv2.imshow('PRO POSTUR ANALIZI', image)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('d'):
        DEBUG_MODE = not DEBUG_MODE
        winsound.Beep(900, 80)

    if key == ord('r'):
        calibrated = False
        monitored_frames = 0
        good_frames = 0

        buffer_eye.clear()
        buffer_shw.clear()
        buffer_slope.clear()
        buffer_nose.clear()
        buffer_facez.clear()
        graph_data.clear()

        for k2 in error_streak:
            error_streak[k2] = 0
            last_evidence_time[k2] = 0

        winsound.Beep(500, 300)

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

if calibrated:
    log_to_excel("SONUC", f"Oturum Bitti. Ortalama Skor: {score}", skor=score)
    print(f"✅ Detaylı rapor '{RAPOR_DOSYASI}' dosyasına kaydedildi.")


✅ SİSTEM HAZIR. Kanıtlar 'Kanit_Fotograflari' klasörüne kaydedilecek.
📸 Kanıt Kaydedildi: Kanit_Fotograflari/20251224_121113_Yana_Egilme.jpg
📸 Kanıt Kaydedildi: Kanit_Fotograflari/20251224_121122_Cok_Yakin.jpg
📸 Kanıt Kaydedildi: Kanit_Fotograflari/20251224_121145_Yana_Egilme.jpg
📸 Kanıt Kaydedildi: Kanit_Fotograflari/20251224_121156_Cok_Yakin.jpg
📸 Kanıt Kaydedildi: Kanit_Fotograflari/20251224_121219_Boyun_Egilme.jpg
📸 Kanıt Kaydedildi: Kanit_Fotograflari/20251224_121229_Yana_Egilme.jpg
✅ Detaylı rapor 'Detayli_Postur_Raporu.csv' dosyasına kaydedildi.
